Descarga de datasets de texto de TF datasets --- 0:00 min
===

* Última modificación: Marzo 1, 2022 | YouTube

Importación de librerías
---

In [4]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# Training set.
train_ds = tfds.load(
    "imdb_reviews",
    split="train[:80%]",
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True,
)

In [ ]:
# Validation set.
val_ds = tfds.load(
    "imdb_reviews",
    split="train[80%:]",
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True,
)

In [ ]:
for review_batch, label_batch in val_ds.take(1):
    for i in range(5):
        print("Review: ", review_batch[i].numpy())
        print("Label: ", label_batch[i].numpy())

In [ ]:
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode="int", output_sequence_length=MAX_SEQUENCE_LENGTH
)

# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = train_ds.map(lambda text, labels: text)
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [ ]:
model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=1)
model.summary()

In [ ]:
model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=1)
model.summary()

In [ ]:
model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
)

In [ ]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

In [ ]:
export_model = tf.keras.Sequential(
    [vectorize_layer, model, layers.Activation("sigmoid")]
)

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer="adam",
    metrics=["accuracy"],
)

In [ ]:
# 0 --> negative review
# 1 --> positive review
inputs = [
    "This is a fantastic movie.",
    "This is a bad movie.",
    "This movie was so bad that it was good.",
    "I will never say yes to watching this movie.",
]

predicted_scores = export_model.predict(inputs)
predicted_labels = [int(round(x[0])) for x in predicted_scores]

for input, label in zip(inputs, predicted_labels):
    print("Question: ", input)
    print("Predicted label: ", label)